# Multi-Face identification using CNN


 - In this notebook we will build our facial identification model step by step to recognize four people. Note that we have images in **jpg** format, and we will go throw the details on how to prepare the data (X, y) for our CNN model. We add an extra folder where we put images for some people that we don't know to classify them as **unknown**.
 - The model will be trained on images containing faces, what I mean by that is each image contains just the face area. We don't take the background into consideration.
 - If your images are not like that, you have two options:
     - Manually resized them and keep just the face part.
     - Use <a href="https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html">face cascade classifier</a> to extract the face part from each image and save them. If the face cascade classifier miss some faces, then you will manually resize them. 

 - The model will run on real-time, for each video frame, we will first extract the faces using face cascade and then make predictions. For each face in an image we make prediction.
 
First let's import some libraries :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from random import randrange, uniform

    
   - We will set the image shape to 32x32. It is too small because the face does not take much space in an image.
   - DATACROP : directory where we have five folders, each one contains images for each person. The 5th folder is for the unknown people. You can dowload images from Kaggle for example.
   - CATEGORIES : name of the five folders(people eg: Said, Blake ... and unknown for unknown people).
   - One example: **"../DATACROP/SAID/hello.jpg"**.

In [ ]:
IMG_SIZE = 32
DATACROP = "dataset_crop"
CATEGORIES = ["AL JADD", "Nossaiba", "EL NABAOUI", "YE", "unknown"]

In [ ]:
data = []
for category in CATEGORIES:
    path = os.path.join(DATACROP,category)
    class_num = CATEGORIES.index(category) 
    for img in tqdm(os.listdir(path)):
        img_path = os.path.join(path,img)
        img_gray = cv2.imread(img_path, 0) 
        img_resized = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))
        data.append([img_resized, class_num])  

- Let's shuffle the data

In [ ]:
random.shuffle(data)

- Let's make an array that contains our images.
- data contains (X, y).

$$X = \begin{bmatrix}---- x1 ----\\
---- x2 ----\\
.\\
.\\
---- xm ----\\
\end{bmatrix}$$

- Where **m** is the total number of images. Let's take an x1 for example. The shape of x1 is **(32, 32, 1)** because we set **IMG_SIZE** = 32 and the number 1 in the 3rd component becasue it's on **grayscale**.

$$y = \begin{bmatrix}0\\
1\\
2\\
3\\
4\\
\end{bmatrix}$$

In [ ]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)


print(f'X shape is :{X.shape} \ny shape is :{y.shape}')

 - **Let's split our data, remember to specify stratify=y so that the classes percentage in train set will be the same in the test set.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
y_train.shape

- If you want to count the percentage of a classe on the training or the test set:

In [ ]:
def count(Y, element):
    i = 0
    for y in Y:
        y = np.array(y)
        element = np.array(element)
        if y == element:
            i += 1
        else:
            pass
    print(f"Percentage of the class {element} is {np.round(100*i/len(Y))}")

In [ ]:
count(y_train, 0)

### One hot for outputs

 - This is very important to understand. In binary classification the y outputs takes two values (0, 1) but, in our case we have five values because class_num vary from 0 to 4. 
 - y is :
$$y = \begin{bmatrix}0\\
1\\
2\\
3\\
4\\
\end{bmatrix}$$

- Our CNN model accept only one hot format which is :
$$y_{one-hot} = \begin{bmatrix}1 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 1\\
\end{bmatrix}$$

- Let's convert y to one hot format be using **to_categorical**:

In [ ]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
y_train

### Data augmentation

   - **Data augmentation** in a technique used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data.
   - Do not add soo much noise so that the model underfit the data.
   - We will use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator">ImageDataGenerator</a> from tensorflow.

In [ ]:
gen = ImageDataGenerator(
        # Rotate images by 40°
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        # shearing the image 
        shear_range=0.1,
        channel_shift_range = 23,
        brightness_range=(0.9, 1.5),
        zoom_range=0.1,
        horizontal_flip=True,
        # The area left after rotationg image will be filled with same color on image
        fill_mode='nearest')

- If you want to display every image along with new created images :


In [ ]:
def show(img1, img2, index):
    w = 10
    h = 10
    fig = plt.figure(figsize=(8, 8))
    columns = 2
    rows = 1
    fig.add_subplot(rows, columns, 1)
    plt.imshow(img1)
    plt.title(f'Full quality {index}')
    fig.add_subplot(rows, columns, 2)
    plt.imshow(img2)
    plt.title(f'Low quality {index}')
    plt.show()

- Set the varable **l** to any number you want. It will determines how many new images will be created for each original image.

In [ ]:
# Try changing it:
l = 20

j = 1
index = 0
data_aug = []
for img in X_train:
    print(f"Image number {j} out of {(l+1)*len(X_train)}", end='\r')
    img_expanded = np.expand_dims(img,0)
    aug_iter = gen.flow(img_expanded)
    aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(l)]
    j += 1
    for image in aug_images:
        s = image
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
        data_aug.append([image, y_train[index]])
        # Show original image and the new created image. This is just to see is things are going well. 
        # Uncomment the following line of code if you want.
        # show(img, image, y_train[index])
        j += 1
    index += 1

### Shuffle the data again

  - Remember to shuffle the data so that the model gets diffrent classes while training.

In [ ]:
random.shuffle(data_aug)

### Save data as pickle format

 - Always save your data so next time you will just load X and y without repeating the above codes again 

In [ ]:
X_train = []
y_train = []

for features,label in data_aug:
    X_train.append(features)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Rescale for fast computation (Backpropagation)
X_train = X_train / 255.0
X_test = X_test / 255.0


# I created a folder name data.pickle to save my data in
pickle_out = open("data.pickle/X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("data.pickle/y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

pickle_out = open("data.pickle/X_test.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("data.pickle/y_test.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

### Load the data

In [ ]:
pickle_in = open("data.pickle/X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("data.pickle/y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("data.pickle/X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("data.pickle/y_test.pickle","rb")
y_test = pickle.load(pickle_in)

### Build the CNN model

In [ ]:
model = Sequential()
# The input shape is the image shape (a, b, 1) 'It is grayscale'
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Dropout(0.7))
model.add(Conv2D(64, kernel_size=3, activation='relu'))

model.add(Conv2D(32, kernel_size=3, activation='relu'))

model.add(Dropout(0.5))


model.add(Flatten())

# I have five classes, so make sure to use the right number bellow
model.add(Dense(5, activation='softmax'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Train the model

 - Epochs are number of passes on the entire dataset. The model will pass 5 times on the dataset. In each epoch, it will takes 20 images (batch_size)

In [ ]:
model.fit(X_train, y_train, batch_size=20, epochs=3, validation_split=0.4)

### Evaluate the model on the test set

In [ ]:
model.evaluate(X_test, y_test)

# If the model suffer from overfitting or underfitting, try changing your CNN model architecture by adding Dropout layers or 
# more hidden layers... 
# You may spend a lot of time retraining your model. You have to know what your model suffer from, 
# and try to solve the problem. This is what deep learning looks like!

### Save the model in h5 format

 - Always save !!!

In [ ]:
# I'll save my model in folder called model_h5_format
model.save("model_h5_format/my_model.h5")

### Load the model

In [ ]:
new_model = load_model("model_h5_format/my_model.h5")
new_model.summary()

In [ ]:
# Make prediction from X_test:
# I will set threshold = 0.5 so if the model gives for example 0.4 probability that an image belongs to a class, I will consider
# not taking it into consideration
def predict_from_X(i, threshold):
    x = X_test[i]
    plt.imshow(x)
    x = np.expand_dims(x, axis=0)
    prediction = new_model.predict(x)
    i  =  np.array(np.where(((prediction >= threshold).astype('int32')[0]) == 1))[0]
    who = ""
    
    if not i.shape == (0,):
        if i[0] == 0:
                who = "AL JADD"

        elif i[0] == 1:
            who = "Nossaiba"

        elif i[0] == 2:
            who = "EL NABAOUI"

        elif i[0] == 3:
            who = "YE Langze"
        else:
            who = "Unknown"
    else:
        who = "Unknown"
    print(who)
    plt.show()

In [ ]:
predict_from_X(randrange(0, X_test.shape[0]-1), threshold=0.9)

## Real time test

In [ ]:
# Make prediction from image (jpg format ...):
def predict_from_frame(image, threshold):
    image = np.expand_dims(image, axis=0)
    prediction = new_model.predict(image)
    i  =  np.array(np.where(((prediction >= threshold).astype('int32')[0]) == 1))[0]
    who = ""
    if not i.shape == (0,):
        if i[0] == 0:
                who = "AL JADD"

        elif i[0] == 1:
            who = "Nossaiba"

        elif i[0] == 2:
            who = "EL NABAOUI"

        elif i[0] == 3:
            who = "YE Langze"
        else:
            who = "Unknown"
    else:
        who = "Unknown"
    return who

In [ ]:
from time import time
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
last_time = time()
face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_default.xml")
i = 1413
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=2, minNeighbors=1)
    for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_sav = frame[y:y+h, x:x+w]
            resized_roi_gray = cv2.resize(roi_gray, (IMG_SIZE, IMG_SIZE))
            # When you wanna make prediction, you have to rescale the image first. I pass two days figuring out why my model
            # outputing the same result. As the model was trained on rescaled images, the prediction must done on rescaled
            # images too.
            resized_roi_gray_recaled = resized_roi_gray/255.0
            
            color = (255, 0, 0) 
            stroke = 2
            end_cord_x = x + w
            end_cord_y = y + h 
            cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
            
            p = predict_from_frame(resized_roi_gray_recaled, threshold=0.8)   
            cv2.putText(frame, p , (x,y), font, 1, (0, 0, 255), 1,1)
           
        
          
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()